In [1]:
import requests
import lxml.html as lh
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
#goal is to enter the IP address we want to investigate,
#hit the submit button on ipvoid,
#grab the results of the table,
#print them out for visualizing,
#rinse and repeat with other websites.
#sample IP address: 213.127.59.231

In [2]:
def submitIPVoid(url, IP):
  #Create a handle, page, to handle the contents of the website
  r = requests.post(url, data={'ip': IP})
  if(r.status_code == 200):
    #Store the contents of the website under doc
    doc = lh.fromstring(r.content)
    #Parse data that are stored between <td>..</td> of HTML
    td_elements = doc.xpath('//td')
    #Create empty list
    col=[]
    for i in range(0,24,2):
      name = td_elements[i].text_content()
      data = td_elements[i+1].text_content()
      col.append((name,[data]))  
    Dict = {title:column for (title,column) in col}
    if(col[2][0].find('POSSIBLY SAFE')):
      print("IPVoid: Possibly Safe IP address")
    elif(col[2][0].find('BLACKLISTED')):
      print("IPVoid: NOT SAFE! BLACKLIST IP IMMEDIATELY!")
    else:
      print("IPVoid: " + col[2])
  else:
    print("IPVoid: " + r.status_code, r.reason)

def submitVT(url):
  #insert api key and create header var
  api = "secretValue"
  headers = {'x-apikey':api}
  #request url with header to provide authentication
  r = requests.get(url, headers=headers)
  #print(r.content) #print the whole html code
  if(r.status_code == 200):
    results = r.json()
    #print(results['data']) #print full json
    print("VirusTotal: " + str(results['data']['attributes']['last_analysis_stats']))
  else:
    print("VirusTotal: " + r.status_code, r.reason)
    
def submitMX(url,driver):
  #Post IP address to load MX Blacklist check
  driver.get(url)
  driver.maximize_window()
  window_before = driver.window_handles[0]
  driver.switch_to.window(window_before)
  driver.implicitly_wait(10) # seconds
  html = driver.page_source
  #Store the contents of the website under doc
  page_content = BeautifulSoup(html, "html.parser")
  elem = driver.find_element(By.CLASS_NAME,"tool-result-body").text
  print("MXToolbox: " + elem)

def submitCisco(url,driver):
  #Post IP address to load MX Blacklist check
  driver.get(url)
  html = driver.page_source
  #Store the contents of the website under page_content
  page_content = BeautifulSoup(html, "html.parser")
  print("Cisco Talos: " + str(page_content))
  driver.implicitly_wait(10) # seconds
  print("********************************")
  window_after = driver.window_handles[1]
  driver.switch_to.window(window_after)
  elem = driver.find_element(By.CLASS_NAME,"lookup-data").text
  elem.click()
    
def main():
  # define the IP address that needs to be investigated
  IP = "213.127.59.232"
  # define a variable to hold the source URL, might make into an array for multiple scans
  url = "https://www.ipvoid.com/ip-blacklist-check/"
  submitIPVoid(url,IP)
  url = "https://www.virustotal.com/api/v3/ip_addresses/" + IP
  submitVT(url)
  driver = webdriver.Firefox()
  url = "https://mxtoolbox.com/SuperTool.aspx?action=blacklist%3a" + IP + "&run=toolpage"
  submitMX(url,driver)
  url = "https://talosintelligence.com/reputation_center/lookup?search=" + IP
  submitCisco(url,driver)
  driver.close()

In [3]:
if __name__ == "__main__":
  main()

IPVoid: Possibly Safe IP address
VirusTotal: {'harmless': 83, 'malicious': 0, 'suspicious': 0, 'undetected': 0, 'timeout': 0}


NoSuchElementException: Message: Unable to locate element: .tool-result-body


In [ ]:
#Sample code previously used
#I'm paranoid and keep old code in case I need an example of what previously worked.
  col=[]
  for i in range(0,24,2):
    name=div_elements[i].text_content()
    data = div_elements[i+1].text_content()
    col.append((name,[data]))  
  Dict={title:column for (title,column) in col}
  print(Dict)
  #for i in range(0,12):
  #  print(col[i])
  if(col[2][0].find('POSSIBLY SAFE')):
    print("Possibly Safe IP address")
  elif(col[2][0].find('BLACKLISTED')):
    print("NOT SAFE! BLACKLIST IP IMMEDIATELY!")
  else:
    print(col[2])
    
   #Parse data that are stored between <td>..</td> of HTML
    #r = requests.post(url, data={'ctl00$ContentPlaceHolder1$txtInput2': "blacklist:"+IP, 'type':'text'},timeout=(3.05,27))
    #r = requests.get(url, timeout=None)
    #print(r.content) #print the whole html code
    #doc = lh.fromstring(r.content)
  
    td_elements = doc.xpath('//span[@id="lblResult"]')
    print(td_elements)
    print("********************")
    div_elements = doc.xpath('//span')
    print(div_elements)
    for i in range(0,16):
      print(div_elements[i].text_content())
    print("********************")
    div_elements = doc.xpath('//@class')
    print(div_elements)
    print("********************")
    div_elements = doc.xpath('//div[@id="callout"]')
    print(div_elements)
    print(td_elements[0].text_content())